**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Outside we must switch on ingress

`minikube addons enable ingress`

In [1]:
kubectl get pods -A | grep --color=always -E '$|ingress'

NAMESPACE       NAME                                        READY   STATUS      RESTARTS   AGE
ingress-nginx   ingress-nginx-admission-create--1-4r566     0/1     Completed   0          46s
ingress-nginx   ingress-nginx-admission-patch--1-p9jbg      0/1     Completed   0          46s
ingress-nginx   ingress-nginx-controller-69bdbc4d57-dtkpc   1/1     Running     0          46s
kube-system     coredns-78fcd69978-jtj8x                    1/1     Running     0          23h
kube-system     etcd-minikube                               1/1     Running     0          23h
kube-system     kube-apiserver-minikube                     1/1     Running     0          23h
kube-system     kube-controller-manager-minikube            1/1     Running     0          23h
kube-system     kube-proxy-7zcnf                            1/1     Running     0          23h
kube-system     kube-scheduler-minikube                     1/1     Running     0          23h
kube-system     storage-provisioner               

In [2]:
kubectl get svc -A

NAMESPACE       NAME                                 TYPE           CLUSTER-IP       EXTERNAL-IP     PORT(S)                      AGE
default         kubernetes                           ClusterIP      10.96.0.1        <none>          443/TCP                      23h
default         nginx-strict                         LoadBalancer   10.108.215.42    10.108.215.42   80:31910/TCP                 18h
ingress-nginx   ingress-nginx-controller             NodePort       10.111.244.238   <none>          80:31565/TCP,443:32485/TCP   2m27s
ingress-nginx   ingress-nginx-controller-admission   ClusterIP      10.102.159.32    <none>          443/TCP                      2m27s
kube-system     kube-dns                             ClusterIP      10.96.0.10       <none>          53/UDP,53/TCP,9153/TCP       23h


In [ ]:
kubectl create -f - <<EOF
apiVersion: apps/v1 
kind: Deployment
metadata:
  name: nginx-deployment
  labels:
    app: nginx
spec:
  replicas: 3
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx
        ports:
        - containerPort: 80
EOF

In [ ]:
kubectl get all

In [ ]:
kubectl create -f - <<EOF
kind: Service
apiVersion: v1
metadata:
  name: "nginx"
spec:
  selector:
    app: "nginx"
  ports:
    - protocol: "TCP"
      port: 80
      targetPort: 80
  type: LoadBalancer
EOF

In [ ]:
kubectl get svc

In [ ]:
kubectl describe service nginx

In [ ]:
kubectl get pods --all-namespaces

In [ ]:
kubectl get svc --all-namespaces

In [ ]:
kubectl get configmap -n kube-system 

In [ ]:
kubectl get configmap -n kube-system  ingress-controller-leader-nginx -o yaml

In [ ]:
kubectl get configmap -n kube-system  nginx-load-balancer-conf -o yaml